In [106]:
from openai import OpenAI
from tqdm import tqdm

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="sk-or-v1-..."
)

def get_batch_sentiment(texts, model="anthropic/claude-3.5-sonnet"):
    """
    Отправляем список новостей и получаем список чисел [-1, 1].
    """
    prompt = (
    "Для каждой из следующих новостей оцени её тональность по шкале от 0 до 1. "
    "Где 0 — исключительно негативная новость, 0.5 — нейтральная, 1 — исключительно позитивная. "
    "Не добавляй текст и пояснения, верни только числа, разделённые запятыми, в том же порядке.\n\n"
    )

    for i, t in enumerate(texts, 1):
        prompt += f"{i}) {t}\n"

    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    result = resp.choices[0].message.content.strip()
    try:
        scores = [float(x.strip()) for x in result.split(",")]
    except:
        scores = [0.0] * len(texts)
    return scores

def add_sentiment(df, text_col="title", batch_size=50, save_every=20, save_path="../data/raw/participants/news_sentiment.csv"):
    sentiments = []
    texts = df[text_col].astype(str).tolist()

    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        batch_scores = get_batch_sentiment(batch)
        sentiments.extend(batch_scores)

        if (i // batch_size + 1) % save_every == 0:
            temp_df = df.iloc[:len(sentiments)].copy()
            temp_df["sentiment"] = sentiments
            temp_df.to_csv(save_path, index=False)
            print(f"[SAVE] Прогресс сохранён на {len(sentiments)} новостях")

    df["sentiment"] = sentiments
    df.to_csv(save_path, index=False)
    return df

news = pd.read_csv("../data/raw/participants/news_2.csv")
if "Unnamed: 0" in news.columns:
    news = news.drop(columns=["Unnamed: 0"])
news["publish_date"] = pd.to_datetime(news["publish_date"])
news["date"] = news["publish_date"].dt.date

news = add_sentiment(news, text_col="title", batch_size=50)

print(news.head())

 49%|████▉     | 20/41 [02:02<02:02,  5.83s/it]

[SAVE] Прогресс сохранён на 1000 новостях


 98%|█████████▊| 40/41 [04:01<00:05,  5.99s/it]

[SAVE] Прогресс сохранён на 2000 новостях


100%|██████████| 41/41 [04:06<00:00,  6.00s/it]

         publish_date                                              title  \
0 2025-04-15 20:11:01  Прибыль Ericsson за 1 квартал превзошла прогно...   
1 2025-04-16 00:09:50  Предварительный просмотр доходов Netflix: Брил...   
2 2025-04-16 03:12:00  AT&T (T) Выигрывает при падении Рынка: Что Вам...   
3 2025-04-16 08:33:16  Российский рынок растет на утренних торгах в о...   
4 2025-04-16 08:57:33  Без драйверов для роста коррекция может законч...   

                                         publication        date  sentiment  
0  Ericsson ERIC сообщила о неоднозначных результ...  2025-04-15        0.8  
1  На фоне волатильности Уолл-стрит большинство а...  2025-04-16        0.7  
2  AT&T (T) завершила недавнюю торговую сессию на...  2025-04-16        0.7  
3  Российский рынок растет во время утренней сесс...  2025-04-16        0.6  
4  Рубль превзошел золото по доходности и стал лу...  2025-04-16        0.3  


In [108]:
import pandas as pd
import re

news = pd.read_csv("../data/raw/participants/news_2.csv")
news = news.drop(columns=["Unnamed: 0"], errors="ignore")
news["publish_date"] = pd.to_datetime(news["publish_date"], errors="coerce")
news["date"] = news["publish_date"].dt.date

tickers = ['GAZP','CHMF','PHOR','MAGN','MGNT','MTSS',
           'PLZL','SBER','VTBR','LKOH','ALRS','AFLT',
           'MOEX','NVTK','ROSN','GMKN','RUAL','T']

company_map = {
    "газпром": "GAZP", "газ": "GAZP",
    "северсталь": "CHMF",
    "фосагро": "PHOR",
    "ммк": "MAGN", "магнитогорский металлургический комбинат": "MAGN",
    "магнит": "MGNT",
    "мтс": "MTSS",
    "полюс": "PLZL",
    "сбер": "SBER", "сбербанк": "SBER",
    "втб": "VTBR",
    "лукойл": "LKOH",
    "алроса": "ALRS",
    "аэрофлот": "AFLT",
    "мосбиржа": "MOEX", "московская биржа": "MOEX",
    "новатэк": "NVTK",
    "роснефть": "ROSN",
    "норникель": "GMKN", "гмк норильский никель": "GMKN",
    "русал": "RUAL",
    "татнефть": "T", "тат": "T"
}

def extract_tickers(row, company_map, n_words=1500):
    title = str(row.get("title", "")).lower()
    publication = str(row.get("publication", "")).lower()
    first_words = " ".join(publication.split()[:n_words])
    text = f"{title} {first_words}"

    found = []
    for name, ticker in company_map.items():
        pattern = r"\b" + re.escape(name) + r"\w*"
        if re.search(pattern, text):
            found.append(ticker)
    return list(set(found))

news["tickers_found"] = news.apply(lambda r: extract_tickers(r, company_map), axis=1)
def attach_sentiment_inplace(news, sentiment_csv_path, sentiment_col="sentiment"):
    import pandas as pd

    sdf = pd.read_csv(sentiment_csv_path)

    sdf["publish_date"] = pd.to_datetime(sdf["publish_date"], errors="coerce")
    news["publish_date"] = pd.to_datetime(news["publish_date"], errors="coerce")

    if sentiment_col not in news.columns:
        news[sentiment_col] = pd.NA

    merged = news.merge(
        sdf[["publish_date", "title", sentiment_col]],
        on=["publish_date", "title"],
        how="left",
        validate="many_to_one",
        suffixes=("", "_new")
    )
    newcol = f"{sentiment_col}_new"
    if newcol in merged.columns:
        merged[sentiment_col] = merged[newcol].combine_first(merged[sentiment_col])
        merged = merged.drop(columns=[newcol])

    return merged
news = attach_sentiment_inplace(news, "../data/raw/participants/news_sentiment.csv", sentiment_col="sentiment")
df_news = news[["publish_date", "date", "tickers_found", "sentiment"]].copy()
import ast
def _to_list(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    if isinstance(x, str):
        x = x.strip()
        if x.startswith("[") and x.endswith("]"):
            try:
                v = ast.literal_eval(x)
                return v if isinstance(v, list) else [str(v)]
            except Exception:
                return [x]
        return [x]
    return [x]

df_news["tickers_found"] = df_news["tickers_found"].apply(_to_list)

allowed = set(tickers)
df_news["tickers_found"] = df_news["tickers_found"].apply(
    lambda lst: [t for t in lst if t in allowed]
)

df_long = df_news.explode("tickers_found")
df_long = df_long.rename(columns={"tickers_found": "ticker"})
df_long = df_long[df_long["ticker"].notna()].reset_index(drop=True)
df_long = df_long.sort_values(["ticker", "publish_date"]).reset_index(drop=True).drop('publish_date', axis=1)
df_long.head()

/tmp/ipykernel_11870/4177933585.py:75: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  merged[sentiment_col] = merged[newcol].combine_first(merged[sentiment_col])


,date,ticker,sentiment
0,2025-04-16,AFLT,0.6
1,2025-04-18,AFLT,0.5
2,2025-04-22,AFLT,0.8
3,2025-04-24,AFLT,0.6
4,2025-04-27,AFLT,0.7
